In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

# Fractal space definition
def lorenz(t, state, sigma=10, beta=2.667, rho=28):
    """
    Lorenz attractor dynamics for chaotic behavior.
    """
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return [dx, dy, dz]

# Fractal computation simulation
def fractal_compute(input_data, iterations=1000):
    """
    Perform chaotic computation in fractal space with dynamic parameters.
    """
    results = []
    for i in range(len(input_data)):
        # Dynamic parameters for each input
        sigma = 10 + np.random.uniform(-2, 2)
        beta = 2.667 + np.random.uniform(-0.5, 0.5)
        rho = 28 + np.random.uniform(-5, 5)

        # Initial state derived from input with noise
        initial_state = [input_data[i] + np.random.normal(0, 0.1),
                         input_data[i] / 2 + np.random.normal(0, 0.1),
                         input_data[i] / 3 + np.random.normal(0, 0.1)]

        # Chaotic evolution in fractal space
        sol = solve_ivp(
            lorenz,
            [0, iterations],
            initial_state,
            t_eval=np.linspace(0, iterations, 100),
            args=(sigma, beta, rho)
        )
        chaotic_result = sol.y[:, -1]  # Final state
        results.append(np.mean(chaotic_result))  # Collapse state
    return results

# Hidden dimension setup
def hidden_dimension_compute(input_data):
    """
    Embed computation into a chaotic hidden dimension.
    """
    # Normalize input for fractal space
    norm_data = np.tanh(input_data)

    # Fractal space computation
    fractal_results = fractal_compute(norm_data)

    # Collapse fractal results back to standard space
    return np.array(fractal_results)

# Example usage
if __name__ == "__main__":
    # Simulated input data
    input_data = np.random.rand(10) * 100  # Random inputs

    # Perform hidden dimension computation
    results = hidden_dimension_compute(input_data)
    print("Results from chaotic time computation:", results)

Results from chaotic time computation: [-0.37511952  2.34280433  9.51295896 -2.04751708  4.79721081  6.00045353
  9.37811958  8.44037648  2.47638762  8.48944073]


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.integrate import solve_ivp
from torch.optim import RAdam
from sklearn.preprocessing import MinMaxScaler

# ----- Chaotic Layer Definition -----
class ChaoticLayer(nn.Module):
    def __init__(self, iterations=500, dt=0.01):
        super(ChaoticLayer, self).__init__()
        self.iterations = iterations
        self.dt = dt

    def forward(self, x):
        # Lorenz system
        def lorenz(t, state, sigma=10, beta=8/3, rho=28):
            x, y, z = state
            dx = sigma * (y - x)
            dy = x * (rho - z) - y
            dz = x * y - beta * z
            return [dx, dy, dz]

        # We'll collect several features from the second half of the trajectory
        results = []
        for sample in x:
            # Initialize with x, y, and R = sqrt(x^2 + y^2)
            initial_state = [
                sample[0].item(),
                sample[1].item(),
                np.sqrt(sample[0].item()**2 + sample[1].item()**2),
            ]

            sol = solve_ivp(
                lorenz,
                [0, self.iterations * self.dt],
                initial_state,
                method='RK45',
                t_eval=np.linspace(0, self.iterations * self.dt, self.iterations)
            )

            # Grab second half
            halfway = sol.y.shape[1] // 2
            x_vals = sol.y[0, halfway:]
            y_vals = sol.y[1, halfway:]
            z_vals = sol.y[2, halfway:]

            # Create multiple chaotic features
            mean_x = np.mean(x_vals)
            mean_y = np.mean(y_vals)
            mean_z = np.mean(z_vals)
            std_x  = np.std(x_vals)
            std_y  = np.std(y_vals)
            std_z  = np.std(z_vals)
            mean_r = np.mean(np.sqrt(x_vals**2 + y_vals**2 + z_vals**2))

            # You can add or remove any stats you find interesting
            chaotic_feature = [
                mean_x,
                mean_y,
                mean_z,
                std_x,
                std_y,
                std_z,
                mean_r
            ]

            results.append(chaotic_feature)

        # Return shape: [batch_size, num_chaotic_features]
        return torch.tensor(results, dtype=torch.float32)

# ----- LCMNet Definition -----
class LCMNet(nn.Module):
    def __init__(self):
        super(LCMNet, self).__init__()

        # Let's expand the initial linear layer to produce more hidden units,
        # and also collect 7 chaotic features from the ChaoticLayer.
        # We'll feed these features into further layers.
        self.fc_input = nn.Linear(2, 16)
        self.bn_input = nn.BatchNorm1d(16)

        self.chaotic = ChaoticLayer(iterations=1000, dt=0.01)

        # Suppose the chaotic layer returns 7 features, we can combine those
        # with the original hidden representation from fc_input. For example,
        # we’ll concatenate them, so we get 16 + 7 = 23 total features.
        self.fc_hidden = nn.Linear(16 + 7, 16)
        self.bn_hidden = nn.BatchNorm1d(16)

        # Final output
        self.fc_out = nn.Linear(16, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        # 1) Transform the raw input
        x_hidden = self.fc_input(x)
        x_hidden = self.relu(self.bn_input(x_hidden))

        # 2) Get chaotic features
        chaotic_feats = self.chaotic(x)  # shape: [batch_size, 7]

        # 3) Concatenate the two representations
        combined = torch.cat([x_hidden, chaotic_feats], dim=1)

        # 4) Another transform
        combined = self.relu(self.bn_hidden(self.fc_hidden(combined)))

        # 5) Final linear layer
        out = self.fc_out(combined)

        # No activation on the final layer (predicting LCM >= 0)
        return out

# ----- LCM Calculation (Baseline for Comparison) -----
def compute_lcm(a, b):
    a, b = int(a), int(b)
    return abs(a * b) // np.gcd(a, b)

# ----- Training and Testing Pipeline -----
if __name__ == "__main__":
    # Synthetic data
    num_samples = 200
    inputs = torch.randint(1, 50, (num_samples, 2)).float()
    lcm_targets = torch.tensor([compute_lcm(a, b) for a, b in inputs.numpy()],
                               dtype=torch.float32).unsqueeze(1)

    # Normalize inputs/targets
    input_scaler = MinMaxScaler()
    inputs = torch.tensor(input_scaler.fit_transform(inputs), dtype=torch.float32)

    target_scaler = MinMaxScaler()
    lcm_targets = torch.tensor(target_scaler.fit_transform(lcm_targets),
                               dtype=torch.float32)

    # Train/test split
    train_size = int(0.8 * num_samples)
    train_inputs, test_inputs = inputs[:train_size], inputs[train_size:]
    train_lcm_targets, test_lcm_targets = lcm_targets[:train_size], lcm_targets[train_size:]

    # Model, loss, optimizer
    model = LCMNet()
    criterion = nn.MSELoss()
    optimizer = RAdam(model.parameters(), lr=0.005)

    # Optional: try a learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)

    # Training loop
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_lcm_targets)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    # Test
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs)
        test_loss = criterion(test_outputs, test_lcm_targets)
        print(f"Test Loss: {test_loss.item():.4f}")

        # Inverse transform
        test_outputs_orig = target_scaler.inverse_transform(test_outputs.numpy())
        test_lcm_targets_orig = target_scaler.inverse_transform(test_lcm_targets.numpy())

        print("\nSample Predictions (Original Scale):")
        for i in range(min(10, len(test_inputs))):
            print(
                f"Input: {test_inputs[i].numpy()}, "
                f"Predicted LCM: {test_outputs_orig[i][0]:.2f}, "
                f"True LCM: {test_lcm_targets_orig[i][0]}"
            )

        # MAPE
        mape = torch.mean(
            torch.abs(
                (torch.tensor(test_lcm_targets_orig) - torch.tensor(test_outputs_orig))
                / torch.tensor(test_lcm_targets_orig)
            )
        ) * 100
        print(f"\nMean Absolute Percentage Error (MAPE): {mape.item():.2f}%")

Epoch 0, Loss: 0.3052
Epoch 50, Loss: 0.0671
Epoch 100, Loss: 0.0174
Epoch 150, Loss: 0.0158
Test Loss: 0.1351

Sample Predictions (Original Scale):
Input: [0.39583334 0.8125    ], Predicted LCM: -261.07, True LCM: 40.0
Input: [0.45833334 0.375     ], Predicted LCM: -648.91, True LCM: 437.0
Input: [0.625     0.5833333], Predicted LCM: -280.49, True LCM: 898.9999389648438
Input: [0.9791667 0.       ], Predicted LCM: -1268.66, True LCM: 47.999996185302734
Input: [0.22916667 0.1875    ], Predicted LCM: -374.50, True LCM: 60.0
Input: [0.0625    0.6458333], Predicted LCM: -290.82, True LCM: 32.0
Input: [0.16666667 0.25      ], Predicted LCM: -560.31, True LCM: 117.0
Input: [0.33333334 0.22916667], Predicted LCM: -436.20, True LCM: 204.0
Input: [0.5833333 0.5833333], Predicted LCM: -327.92, True LCM: 28.999998092651367
Input: [0.8125    0.6666667], Predicted LCM: -198.24, True LCM: 1320.0

Mean Absolute Percentage Error (MAPE): 518.66%


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.integrate import solve_ivp
from torch.optim import RAdam
from sklearn.preprocessing import StandardScaler

# ----- Chaotic Layer Definition -----
class ChaoticLayer(nn.Module):
    def __init__(self, iterations=500, dt=0.01):
        super(ChaoticLayer, self).__init__()
        self.iterations = iterations
        self.dt = dt

    def forward(self, x):
        def lorenz(t, state, sigma=10, beta=8/3, rho=28):
            x, y, z = state
            dx = sigma * (y - x)
            dy = x * (rho - z) - y
            dz = x * y - beta * z
            return [dx, dy, dz]

        results = []
        for sample in x:
            initial_state = [
                sample[0].item(),
                sample[1].item(),
                np.sqrt(sample[0].item()**2 + sample[1].item()**2),
            ]
            sol = solve_ivp(
                lorenz,
                [0, self.iterations * self.dt],
                initial_state,
                method='RK45',
                t_eval=np.linspace(0, self.iterations * self.dt, self.iterations),
            )
            x_vals, y_vals, z_vals = sol.y
            chaotic_features = [
                np.mean(z_vals), np.std(z_vals),
                np.mean(np.sqrt(x_vals**2 + y_vals**2 + z_vals**2))
            ]
            results.append(chaotic_features)

        return torch.tensor(results, dtype=torch.float32)

# ----- LCMNet Definition -----
class LCMNet(nn.Module):
    def __init__(self):
        super(LCMNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.chaotic = ChaoticLayer(iterations=500)
        self.fc2 = nn.Linear(16 + 3, 16)  # Adding chaotic features
        self.fc3 = nn.Linear(16, 1)
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x):
        x_hidden = self.leaky_relu(self.bn1(self.fc1(x)))
        chaotic_features = self.chaotic(x)
        combined = torch.cat([x_hidden, chaotic_features], dim=1)
        x_out = self.leaky_relu(self.fc2(combined))
        x_out = self.fc3(x_out)
        return torch.abs(x_out)  # Ensure positive output

# ----- LCM Calculation -----
def compute_lcm(a, b):
    a, b = int(a), int(b)
    return abs(a * b) // np.gcd(a, b)

# ----- Training and Testing -----
if __name__ == "__main__":
    num_samples = 200
    inputs = torch.randint(1, 50, (num_samples, 2)).float()
    lcm_targets = torch.tensor([compute_lcm(a, b) for a, b in inputs.numpy()], dtype=torch.float32).unsqueeze(1)

    input_scaler = StandardScaler()
    inputs = torch.tensor(input_scaler.fit_transform(inputs), dtype=torch.float32)

    # Do not scale outputs; let the network learn directly
    train_size = int(0.8 * num_samples)
    train_inputs, test_inputs = inputs[:train_size], inputs[train_size:]
    train_lcm_targets, test_lcm_targets = lcm_targets[:train_size], lcm_targets[train_size:]

    model = LCMNet()
    criterion = nn.MSELoss()
    optimizer = RAdam(model.parameters(), lr=0.005)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)

    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_lcm_targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs)
        test_loss = criterion(test_outputs, test_lcm_targets)
        print(f"Test Loss: {test_loss.item():.4f}")

        test_outputs_orig = test_outputs.numpy()
        test_lcm_targets_orig = test_lcm_targets.numpy()

        print("\nSample Predictions:")
        for i in range(10):
            print(f"Input: {test_inputs[i].numpy()}, Predicted: {test_outputs_orig[i][0]:.2f}, True: {test_lcm_targets_orig[i][0]}")

        mape = torch.mean(torch.abs((torch.tensor(test_lcm_targets_orig) - torch.tensor(test_outputs_orig)) / torch.tensor(test_lcm_targets_orig))) * 100
        print(f"\nMean Absolute Percentage Error (MAPE): {mape.item():.2f}%")

Epoch 0, Loss: 355942.9375
Epoch 50, Loss: 353984.7500
Epoch 100, Loss: 350027.9375
Epoch 150, Loss: 343087.4688
Epoch 200, Loss: 331731.2812
Epoch 250, Loss: 323585.5312
Epoch 300, Loss: 313279.0625
Epoch 350, Loss: 300450.4062
Epoch 400, Loss: 284909.7500
Epoch 450, Loss: 275970.5312
Epoch 500, Loss: 266125.5938
Epoch 550, Loss: 255256.8750
Epoch 600, Loss: 243448.4062
Epoch 650, Loss: 237267.0312
Epoch 700, Loss: 230903.6719
Epoch 750, Loss: 224357.7031
Epoch 800, Loss: 217664.2500
Epoch 850, Loss: 214251.0469
Epoch 900, Loss: 210788.0938
Epoch 950, Loss: 207276.5781
Test Loss: 221388.3750

Sample Predictions:
Input: [-0.13298798  0.84966195], Predicted: 227.16, True: 828.0
Input: [ 0.53011    -0.25693887], Predicted: 196.53, True: 160.0
Input: [ 0.45643243 -0.8794018 ], Predicted: 156.89, True: 341.0
Input: [0.16172223 0.36552408], Predicted: 214.15, True: 783.0
Input: [ 0.53011    -0.11861377], Predicted: 205.19, True: 352.0
Input: [0.08804467 0.43468663], Predicted: 214.19, True:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.integrate import solve_ivp
from torch.optim import RAdam
from sklearn.preprocessing import StandardScaler

def compute_lcm(a, b):
    a, b = int(a), int(b)
    return abs(a * b) // np.gcd(a, b)

class EnhancedChaoticLayer(nn.Module):
    def __init__(self, iterations=500, dt=0.01, system='lorenz'):
        super(EnhancedChaoticLayer, self).__init__()
        self.iterations = iterations
        self.dt = dt
        self.system = system

    def _lorenz(self, t, state, sigma=10, beta=8/3, rho=28):
        x, y, z = state
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        return [dx, dy, dz]

    def _rossler(self, t, state, a=0.2, b=0.2, c=5.7):
        x, y, z = state
        dx = -y - z
        dy = x + a * y
        dz = b + z * (x - c)
        return [dx, dy, dz]

    def _chen(self, t, state, a=35, b=3, c=28):
        x, y, z = state
        dx = a * (y - x)
        dy = (c - a) * x - x * z + c * y
        dz = x * y - b * z
        return [dx, dy, dz]

    def _extract_features(self, trajectory):
        x_vals, y_vals, z_vals = trajectory

        # Basic statistical features
        means = [np.mean(vals) for vals in [x_vals, y_vals, z_vals]]
        stds = [np.std(vals) for vals in [x_vals, y_vals, z_vals]]

        # Dynamic features
        radius = np.sqrt(x_vals**2 + y_vals**2 + z_vals**2)
        angular_velocity = np.diff(np.arctan2(y_vals[:-1], x_vals[:-1]))

        # Lyapunov-inspired features
        divergence_rate = np.mean(np.abs(np.diff(radius)))

        features = [
            *means, *stds,                      # 6 features
            np.mean(radius), np.std(radius),    # 2 features
            np.mean(np.abs(angular_velocity)),  # 1 feature
            divergence_rate                     # 1 feature
        ]

        return features

    def forward(self, x):
        system_func = {
            'lorenz': self._lorenz,
            'rossler': self._rossler,
            'chen': self._chen
        }[self.system]

        results = []
        for sample in x:
            initial_state = [
                sample[0].item(),
                sample[1].item(),
                np.sqrt(sample[0].item()**2 + sample[1].item()**2)
            ]

            sol = solve_ivp(
                system_func,
                [0, self.iterations * self.dt],
                initial_state,
                method='RK45',
                t_eval=np.linspace(0, self.iterations * self.dt, self.iterations)
            )

            features = self._extract_features(sol.y)
            results.append(features)

        return torch.tensor(results, dtype=torch.float32)

class EnhancedLCMNet(nn.Module):
    def __init__(self):
        super(EnhancedLCMNet, self).__init__()

        # Number of features from chaotic layer (updated)
        chaotic_features = 10  # 3 means + 3 stds + 2 radius stats + angular velocity + divergence

        # Initial processing
        self.fc1 = nn.Linear(2, 32)
        self.bn1 = nn.BatchNorm1d(32)

        # Parallel chaotic systems
        self.chaotic_lorenz = EnhancedChaoticLayer(iterations=500, system='lorenz')
        self.chaotic_rossler = EnhancedChaoticLayer(iterations=500, system='rossler')
        self.chaotic_chen = EnhancedChaoticLayer(iterations=500, system='chen')

        # Combine features
        combined_features = 32 + (chaotic_features * 3)  # Regular features + (3 chaotic systems * features per system)
        self.fc2 = nn.Linear(combined_features, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)

        self.dropout = nn.Dropout(0.1)
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x):
        # Regular path
        x_hidden = self.leaky_relu(self.bn1(self.fc1(x)))

        # Chaotic features from different systems
        lorenz_features = self.chaotic_lorenz(x)
        rossler_features = self.chaotic_rossler(x)
        chen_features = self.chaotic_chen(x)

        # Combine all features
        combined = torch.cat([
            x_hidden,
            lorenz_features,
            rossler_features,
            chen_features
        ], dim=1)

        # Final processing
        x_out = self.leaky_relu(self.bn2(self.fc2(combined)))
        x_out = self.dropout(x_out)
        x_out = self.leaky_relu(self.bn3(self.fc3(x_out)))
        x_out = self.fc4(x_out)

        return torch.abs(x_out)

def train_model(model, train_inputs, train_targets, val_inputs, val_targets, epochs=1000, patience=50):
    criterion = nn.MSELoss()
    optimizer = RAdam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)

    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_targets)

        scheduler.step(val_loss)

        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                model.load_state_dict(best_model_state)
                break

    return model

if __name__ == "__main__":
    # Data preparation
    num_samples = 300  # Increased sample size
    inputs = torch.randint(1, 50, (num_samples, 2)).float()
    lcm_targets = torch.tensor([compute_lcm(a, b) for a, b in inputs.numpy()], dtype=torch.float32).unsqueeze(1)

    # Scaling
    input_scaler = StandardScaler()
    inputs = torch.tensor(input_scaler.fit_transform(inputs), dtype=torch.float32)

    # Split data
    train_size = int(0.7 * num_samples)
    val_size = int(0.15 * num_samples)
    train_inputs, val_inputs, test_inputs = inputs[:train_size], inputs[train_size:train_size+val_size], inputs[train_size+val_size:]
    train_lcm_targets, val_lcm_targets, test_lcm_targets = lcm_targets[:train_size], lcm_targets[train_size:train_size+val_size], lcm_targets[train_size+val_size:]

    # Train model
    model = EnhancedLCMNet()
    model = train_model(model, train_inputs, train_lcm_targets, val_inputs, val_lcm_targets)

    # Evaluation
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs)
        test_loss = nn.MSELoss()(test_outputs, test_lcm_targets)
        print(f"\nTest Loss: {test_loss.item():.4f}")

        test_outputs_np = test_outputs.numpy()
        test_targets_np = test_lcm_targets.numpy()

        print("\nSample Predictions:")
        for i in range(min(10, len(test_inputs))):
            orig_inputs = input_scaler.inverse_transform(test_inputs[i].numpy().reshape(1, -1))[0]
            print(f"Numbers: ({int(orig_inputs[0])}, {int(orig_inputs[1])}), "
                  f"Predicted LCM: {test_outputs_np[i][0]:.1f}, "
                  f"True LCM: {test_targets_np[i][0]}")

        mape = np.mean(np.abs((test_targets_np - test_outputs_np) / test_targets_np)) * 100
        print(f"\nMean Absolute Percentage Error: {mape:.2f}%")

Epoch 0, Train Loss: 511555.9688, Val Loss: 421606.5625
Epoch 50, Train Loss: 511260.7500, Val Loss: 421572.7188
Epoch 100, Train Loss: 510993.5625, Val Loss: 421247.4688
Epoch 150, Train Loss: 510570.0625, Val Loss: 420860.0000
Epoch 200, Train Loss: 510182.7500, Val Loss: 420541.7812
Epoch 250, Train Loss: 509931.1875, Val Loss: 420270.2812
Epoch 300, Train Loss: 509605.3438, Val Loss: 419992.5312
Epoch 350, Train Loss: 509379.8750, Val Loss: 419830.0312
Epoch 400, Train Loss: 509160.1562, Val Loss: 419684.0000
Epoch 450, Train Loss: 508971.9375, Val Loss: 419548.1875
Epoch 500, Train Loss: 508686.5625, Val Loss: 419371.1250
Epoch 550, Train Loss: 508480.4062, Val Loss: 419242.5312
Early stopping triggered at epoch 594

Test Loss: 438579.1250

Sample Predictions:
Numbers: (45, 14), Predicted LCM: 0.4, True LCM: 630.0
Numbers: (5, 22), Predicted LCM: 1.5, True LCM: 66.0
Numbers: (6, 10), Predicted LCM: 0.5, True LCM: 77.0
Numbers: (17, 9), Predicted LCM: 0.6, True LCM: 153.0
Numbers: 

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.integrate import solve_ivp
from torch.optim import RAdam
from sklearn.preprocessing import StandardScaler
import math

def compute_lcm(a, b):
    a, b = int(a), int(b)
    return abs(a * b) // np.gcd(a, b)

class EnhancedChaoticLayer(nn.Module):
    def __init__(self, iterations=200, dt=0.01, system='lorenz'):
        super(EnhancedChaoticLayer, self).__init__()
        self.iterations = iterations
        self.dt = dt
        self.system = system

    def _lorenz(self, t, state, sigma=10, beta=8/3, rho=28):
        x, y, z = state
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        return [dx, dy, dz]

    def _rossler(self, t, state, a=0.2, b=0.2, c=5.7):
        x, y, z = state
        dx = -y - z
        dy = x + a * y
        dz = b + z * (x - c)
        return [dx, dy, dz]

    def _extract_features(self, trajectory):
        x_vals, y_vals, z_vals = trajectory

        # Basic statistical features
        means = [np.mean(vals) for vals in [x_vals, y_vals, z_vals]]
        stds = [np.std(vals) for vals in [x_vals, y_vals, z_vals]]

        # Dynamic features
        radius = np.sqrt(x_vals**2 + y_vals**2 + z_vals**2)

        features = [
            *means, *stds,                      # 6 features
            np.mean(radius), np.std(radius),    # 2 features
        ]

        return features

    def forward(self, x):
        system_func = {
            'lorenz': self._lorenz,
            'rossler': self._rossler
        }[self.system]

        results = []
        for sample in x:
            # Scale initial conditions based on input values
            scale_factor = math.log(abs(sample[0].item() * sample[1].item()) + 1)
            initial_state = [
                sample[0].item() * scale_factor,
                sample[1].item() * scale_factor,
                scale_factor
            ]

            sol = solve_ivp(
                system_func,
                [0, self.iterations * self.dt],
                initial_state,
                method='RK45',
                t_eval=np.linspace(0, self.iterations * self.dt, self.iterations)
            )

            features = self._extract_features(sol.y)
            results.append(features)

        return torch.tensor(results, dtype=torch.float32)

class EnhancedLCMNet(nn.Module):
    def __init__(self):
        super(EnhancedLCMNet, self).__init__()

        chaotic_features = 8  # 3 means + 3 stds + 2 radius stats

        # Initial processing of raw inputs
        self.fc1 = nn.Linear(2, 16)
        self.bn1 = nn.BatchNorm1d(16)

        # Process mathematical features
        self.math_fc = nn.Linear(4, 16)  # For GCD-related features
        self.bn_math = nn.BatchNorm1d(16)

        # Chaotic systems
        self.chaotic_lorenz = EnhancedChaoticLayer(iterations=200, system='lorenz')
        self.chaotic_rossler = EnhancedChaoticLayer(iterations=200, system='rossler')

        # Combine features
        combined_features = 16 + 16 + (chaotic_features * 2)
        self.fc2 = nn.Linear(combined_features, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)

        self.dropout = nn.Dropout(0.1)
        self.leaky_relu = nn.LeakyReLU()

    def compute_math_features(self, x):
        # Extract mathematical features that might help with LCM
        a = x[:, 0]
        b = x[:, 1]
        gcd = torch.tensor([math.gcd(int(a[i]), int(b[i])) for i in range(len(a))], dtype=torch.float32)
        product = torch.abs(a * b)
        max_val = torch.max(torch.abs(x), dim=1)[0]
        min_val = torch.min(torch.abs(x), dim=1)[0]

        return torch.stack([gcd, product, max_val, min_val], dim=1)

    def forward(self, x):
        # Process raw inputs
        x_hidden = self.leaky_relu(self.bn1(self.fc1(x)))

        # Process mathematical features
        math_features = self.compute_math_features(x)
        math_hidden = self.leaky_relu(self.bn_math(self.math_fc(math_features)))

        # Get chaotic features
        lorenz_features = self.chaotic_lorenz(x)
        rossler_features = self.chaotic_rossler(x)

        # Combine all features
        combined = torch.cat([
            x_hidden,
            math_hidden,
            lorenz_features,
            rossler_features
        ], dim=1)

        # Final processing
        x_out = self.leaky_relu(self.bn2(self.fc2(combined)))
        x_out = self.dropout(x_out)
        x_out = self.leaky_relu(self.fc3(x_out))
        x_out = self.fc4(x_out)

        return torch.abs(x_out)

def train_model(model, train_inputs, train_targets, val_inputs, val_targets, epochs=1000, patience=50):
    criterion = nn.MSELoss()
    optimizer = RAdam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=True)

    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_targets)

        scheduler.step(val_loss)

        if epoch % 50 == 0:
            train_mape = torch.mean(torch.abs((train_targets - outputs) / train_targets)) * 100
            val_mape = torch.mean(torch.abs((val_targets - val_outputs) / val_targets)) * 100
            print(f"Epoch {epoch}, Train MAPE: {train_mape:.2f}%, Val MAPE: {val_mape:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                model.load_state_dict(best_model_state)
                break

    return model

if __name__ == "__main__":
    # Data preparation
    num_samples = 500
    inputs = torch.randint(1, 50, (num_samples, 2)).float()
    lcm_targets = torch.tensor([compute_lcm(a, b) for a, b in inputs.numpy()], dtype=torch.float32).unsqueeze(1)

    # Log transform the targets
    lcm_targets = torch.log1p(lcm_targets)

    # Scaling
    input_scaler = StandardScaler()
    inputs = torch.tensor(input_scaler.fit_transform(inputs), dtype=torch.float32)

    # Split data
    train_size = int(0.7 * num_samples)
    val_size = int(0.15 * num_samples)
    train_inputs = inputs[:train_size]
    val_inputs = inputs[train_size:train_size+val_size]
    test_inputs = inputs[train_size+val_size:]

    train_targets = lcm_targets[:train_size]
    val_targets = lcm_targets[train_size:train_size+val_size]
    test_targets = lcm_targets[train_size+val_size:]

    # Train model
    model = EnhancedLCMNet()
    model = train_model(model, train_inputs, train_targets, val_inputs, val_targets)

    # Evaluation
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_inputs)

        # Convert back from log space
        test_outputs_original = torch.expm1(test_outputs).numpy()
        test_targets_original = torch.expm1(test_targets).numpy()

        print("\nSample Predictions:")
        for i in range(min(10, len(test_inputs))):
            orig_inputs = input_scaler.inverse_transform(test_inputs[i].numpy().reshape(1, -1))[0]
            print(f"Numbers: ({int(orig_inputs[0])}, {int(orig_inputs[1])}), "
                  f"Predicted LCM: {test_outputs_original[i][0]:.1f}, "
                  f"True LCM: {test_targets_original[i][0]:.1f}")

        mape = np.mean(np.abs((test_targets_original - test_outputs_original) / test_targets_original)) * 100
        print(f"\nMean Absolute Percentage Error: {mape:.2f}%")

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Train MAPE: 96.53%, Val MAPE: 92.59%
Epoch 50, Train MAPE: 96.44%, Val MAPE: 96.31%
Early stopping triggered at epoch 50

Sample Predictions:
Numbers: (2, 4), Predicted LCM: 0.4, True LCM: 15.0
Numbers: (28, 15), Predicted LCM: 0.2, True LCM: 420.0
Numbers: (49, 28), Predicted LCM: 0.1, True LCM: 196.0
Numbers: (12, 45), Predicted LCM: 0.1, True LCM: 180.0
Numbers: (29, 16), Predicted LCM: 0.1, True LCM: 464.0
Numbers: (8, 32), Predicted LCM: 0.3, True LCM: 32.0
Numbers: (34, 3), Predicted LCM: 0.1, True LCM: 102.0
Numbers: (1, 21), Predicted LCM: 0.2, True LCM: 21.0
Numbers: (42, 8), Predicted LCM: 0.1, True LCM: 168.0
Numbers: (17, 13), Predicted LCM: 0.2, True LCM: 221.0

Mean Absolute Percentage Error: 99.56%
